### Training a RL agent on Atari games

In [7]:
make_atari_env??

### CartPole Environment

### Multiprocessing: Unleashing the Power of Vectorized Environments

In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

def make_env(env_id: str, rank: int, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = gym.make(env_id, render_mode="human")
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

# if __name__ == "__main__":

env_id = "CartPole-v1"
num_cpu = 4  # Number of processes to use
# Create the vectorized environment
vec_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

# Stable Baselines provides you with make_vec_env() helper
# which does exactly the previous steps for you.
# You can choose between `DummyVecEnv` (usually faster) and `SubprocVecEnv`
# env = make_vec_env(env_id, n_envs=num_cpu, seed=0, vec_env_cls=SubprocVecEnv)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=50_000)
model.save("models/CartPole")

obs = vec_env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()

Using cuda device


### Callbacks: Evaluate Agent Performance

In [ ]:
import os
import gymnasium as gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env

env_id = "Pendulum-v1"
n_training_envs = 1
n_eval_envs = 5

# Create log dir where evaluation results will be saved
eval_log_dir = "./models/Pendulum_eval_logs/"
os.makedirs(eval_log_dir, exist_ok=True)

# Initialize a vectorized training environment with default parameters
train_env = make_vec_env(env_id, n_envs=n_training_envs, seed=0)

# Separate evaluation env, with different parameters passed via env_kwargs
# Eval environments can be vectorized to speed up evaluation.
eval_env = make_vec_env(env_id, n_envs=n_eval_envs, seed=0,
                        env_kwargs={'g':0.7})

# Create callback that evaluates agent for 5 episodes every 500 training environment steps.
# When using multiple training environments, agent will be evaluated every
# eval_freq calls to train_env.step(), thus it will be evaluated every
# (eval_freq * n_envs) training steps. See EvalCallback doc for more information.
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

model = SAC("MlpPolicy", train_env)
# model = SAC.load("./models/Pendulum_eval_logs/best_model", env=train_env)

model.learn(50000, callback=eval_callback)


model = SAC.load("./models/Pendulum_eval_logs/best_model", env=train_env)

obs = train_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = train_env.step(action)
    train_env.render("human")

Eval num_timesteps=500, episode_reward=-131.90 +/- 121.17
Episode length: 200.00 +/- 0.00
New best mean reward!


#### Pong

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C
import time

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

# model = A2C("CnnPolicy", vec_env, verbose=1)
model = A2C.load("models/Pong", env=vec_env)

model.learn(total_timesteps=50_000)
model.save("models/Pong")

print(f"train time: {time.time()-t0}")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Wrapping the env in a VecTransposeImage.
------------------------------------
| time/                 |          |
|    fps                | 283      |
|    iterations         | 100      |
|    time_elapsed       | 7        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.03    |
|    explained_variance | 0.927    |
|    learning_rate      | 0.0007   |
|    n_updates          | 6349     |
|    policy_loss        | -0.00657 |
|    value_loss         | 0.00536  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.36e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 320      |
|    iterations         | 200      |
|    time_elapsed       | 12       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -1.35    |
|    explained_variance | 0.303   

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.49e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 329      |
|    iterations         | 1500     |
|    time_elapsed       | 90       |
|    total_timesteps    | 30000    |
| train/                |          |
|    entropy_loss       | -1.5     |
|    explained_variance | 0.655    |
|    learning_rate      | 0.0007   |
|    n_updates          | 7749     |
|    policy_loss        | 0.0464   |
|    value_loss         | 0.0167   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 3.48e+03 |
|    ep_rew_mean        | -20.5    |
| time/                 |          |
|    fps                | 329      |
|    iterations         | 1600     |
|    time_elapsed       | 97       |
|    total_timesteps    | 32000    |
| train/                |          |
|

D:\apps\Anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


#### Breakout

In [1]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C
import time

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

model = A2C("CnnPolicy",
            vec_env, 
            tensorboard_log="models/Breakout2_A2C/",
            verbose=1)

# model = A2C.load("models/Breakout", env=vec_env)

model.learn(total_timesteps=10_000)
model.save("models/Breakout2_A2C/model")

print(f"train time: {time.time()-t0}")

# obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs, deterministic=False)
#     obs, rewards, dones, info = vec_env.step(action)
#     vec_env.render("human")
    

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to models/Breakout2_A2C/A2C_2
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 766      |
|    ep_rew_mean        | 1.5      |
| time/                 |          |
|    fps                | 56       |
|    iterations         | 100      |
|    time_elapsed       | 8        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.39    |
|    explained_variance | -0.18    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.0344  |
|    value_loss         | 0.00177  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 756      |
|    ep_rew_mean        | 1.4      |
| time/                 |          |
|    fps                | 77       |
|    iterations         | 200      |
|    time_elaps

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 778       |
|    ep_rew_mean        | 1.62      |
| time/                 |           |
|    fps                | 114       |
|    iterations         | 1400      |
|    time_elapsed       | 61        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -1.39     |
|    explained_variance | -1.07e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1399      |
|    policy_loss        | -0.00447  |
|    value_loss         | 3.33e-05  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 761      |
|    ep_rew_mean        | 1.51     |
| time/                 |          |
|    fps                | 115      |
|    iterations         | 1500     |
|    time_elapsed       | 65       |
|    total_timesteps    | 7500     |
| train/             

In [ ]:
tensorboard --logdir ./models/Breakout2_A2C/A2C_1

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1,
                         seed=1
                        )
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

model = A2C.load("models/Breakout2_A2C/model", env=vec_env)

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")
    

Wrapping the env in a VecTransposeImage.


D:\apps\Anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [ ]:
!tensorboard --logdir tensorboard_log/Breakout2_A2C

In [1]:
from stable_baselines3 import A2C

model = A2C("MlpPolicy", "CartPole-v1", verbose=1, tensorboard_log="./a2c_cartpole_tensorboard/")
model.learn(total_timesteps=10_000)

Using cuda device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./a2c_cartpole_tensorboard/A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 22       |
|    ep_rew_mean        | 22       |
| time/                 |          |
|    fps                | 81       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.692   |
|    explained_variance | -0.0384  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.41     |
|    value_loss         | 6.15     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 22.7     |
|    ep_rew_mean        | 22.7     |
| time/                 |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 51       |
|    ep_rew_mean        | 51       |
| time/                 |          |
|    fps                | 215      |
|    iterations         | 1400     |
|    time_elapsed       | 32       |
|    total_timesteps    | 7000     |
| train/                |          |
|    entropy_loss       | -0.535   |
|    explained_variance | 0.000418 |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | 1        |
|    value_loss         | 1.59     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 53.2     |
|    ep_rew_mean        | 53.2     |
| time/                 |          |
|    fps                | 217      |
|    iterations         | 1500     |
|    time_elapsed       | 34       |
|    total_timesteps    | 7500     |
| train/                |          |
|

In [3]:
!tensorboard --logdir ./a2c_cartpole_tensorboard/A2C_1 --port=6007

^C


In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import PPO
import time

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# env = make_atari_env("BreakoutNoFrameskip-v4", seed=0)
vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, 
#                          seed=0
                        )
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)


model = PPO('CnnPolicy', vec_env,
#               policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=2.5e-4,
              ent_coef=0.01,
              n_steps=128,
              n_epochs=4,
              batch_size=256,
              gamma=0.99,
              gae_lambda=0.95,
              clip_range=0.1,
              vf_coef=0.5,
              verbose=1,
              tensorboard_log="models/Breakout_ppo/")

model = PPO.load("models/Breakout_ppo/model", env=vec_env)

# model.learn(int(2e4))
model.learn(total_timesteps=5000_000)
model.save("models/Breakout_ppo/model")

print(f"train time: {time.time()-t0}")


vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
vec_env = VecFrameStack(vec_env, n_stack=4)
model = PPO.load("models/Breakout_ppo/model", env=vec_env)

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Using cuda device
Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.
Logging to models/Breakout_ppo/PPO_7
----------------------------
| time/              |     |
|    fps             | 104 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 153        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 1024       |
| train/                  |            |
|    approx_kl            | 0.02099319 |
|    clip_fraction        | 0.0806     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.28      |
|    explained_variance   | 0.962      |
|    learning_rate        | 0.00025    |
|    loss                 | -0.0452    |
|    n_updates            | 10488      |
|    policy_gradient_loss | -0.0193    |
|   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.44e+03    |
|    ep_rew_mean          | 19.1        |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 12          |
|    time_elapsed         | 21          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.047737863 |
|    clip_fraction        | 0.0957      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.359      |
|    explained_variance   | 0.803       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.031      |
|    n_updates            | 10528       |
|    policy_gradient_loss | -0.0201     |
|    value_loss           | 0.015       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.44e+

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.77e+03  |
|    ep_rew_mean          | 22.1      |
| time/                   |           |
|    fps                  | 306       |
|    iterations           | 22        |
|    time_elapsed         | 36        |
|    total_timesteps      | 11264     |
| train/                  |           |
|    approx_kl            | 0.0300002 |
|    clip_fraction        | 0.0669    |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.337    |
|    explained_variance   | 0.871     |
|    learning_rate        | 0.00025   |
|    loss                 | -0.0319   |
|    n_updates            | 10568     |
|    policy_gradient_loss | -0.012    |
|    value_loss           | 0.00858   |
---------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.79e+03    |
|    ep_rew_mean          | 22.5  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.72e+03    |
|    ep_rew_mean          | 22.7        |
| time/                   |             |
|    fps                  | 315         |
|    iterations           | 32          |
|    time_elapsed         | 51          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.027341645 |
|    clip_fraction        | 0.0713      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.338      |
|    explained_variance   | 0.884       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0282     |
|    n_updates            | 10608       |
|    policy_gradient_loss | -0.0159     |
|    value_loss           | 0.00788     |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.72e+03

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.7e+03     |
|    ep_rew_mean          | 22.7        |
| time/                   |             |
|    fps                  | 321         |
|    iterations           | 42          |
|    time_elapsed         | 66          |
|    total_timesteps      | 21504       |
| train/                  |             |
|    approx_kl            | 0.025036095 |
|    clip_fraction        | 0.0571      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.324      |
|    explained_variance   | 0.833       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0315     |
|    n_updates            | 10648       |
|    policy_gradient_loss | -0.0142     |
|    value_loss           | 0.0102      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.7e+0

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.85e+03    |
|    ep_rew_mean          | 23.1        |
| time/                   |             |
|    fps                  | 336         |
|    iterations           | 1822        |
|    time_elapsed         | 2771        |
|    total_timesteps      | 932864      |
| train/                  |             |
|    approx_kl            | 0.018907944 |
|    clip_fraction        | 0.0635      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.45       |
|    explained_variance   | 0.925       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0461     |
|    n_updates            | 17768       |
|    policy_gradient_loss | -0.019      |
|    value_loss           | 0.00521     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.84e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.81e+03    |
|    ep_rew_mean          | 22.7        |
| time/                   |             |
|    fps                  | 336         |
|    iterations           | 1832        |
|    time_elapsed         | 2786        |
|    total_timesteps      | 937984      |
| train/                  |             |
|    approx_kl            | 0.032181546 |
|    clip_fraction        | 0.0952      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.504      |
|    explained_variance   | 0.865       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0405     |
|    n_updates            | 17808       |
|    policy_gradient_loss | -0.018      |
|    value_loss           | 0.00878     |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.81e+03

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.82e+03  |
|    ep_rew_mean          | 23        |
| time/                   |           |
|    fps                  | 336       |
|    iterations           | 1842      |
|    time_elapsed         | 2802      |
|    total_timesteps      | 943104    |
| train/                  |           |
|    approx_kl            | 0.0312713 |
|    clip_fraction        | 0.102     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.477    |
|    explained_variance   | 0.94      |
|    learning_rate        | 0.00025   |
|    loss                 | -0.049    |
|    n_updates            | 17848     |
|    policy_gradient_loss | -0.0206   |
|    value_loss           | 0.00402   |
---------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.82e+03    |
|    ep_rew_mean          | 22.8  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.85e+03    |
|    ep_rew_mean          | 22.9        |
| time/                   |             |
|    fps                  | 336         |
|    iterations           | 1852        |
|    time_elapsed         | 2817        |
|    total_timesteps      | 948224      |
| train/                  |             |
|    approx_kl            | 0.030692527 |
|    clip_fraction        | 0.0767      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.521      |
|    explained_variance   | 0.888       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0513     |
|    n_updates            | 17888       |
|    policy_gradient_loss | -0.0205     |
|    value_loss           | 0.00738     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.86e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.8e+03     |
|    ep_rew_mean          | 22.5        |
| time/                   |             |
|    fps                  | 336         |
|    iterations           | 1862        |
|    time_elapsed         | 2833        |
|    total_timesteps      | 953344      |
| train/                  |             |
|    approx_kl            | 0.018324647 |
|    clip_fraction        | 0.0938      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.438      |
|    explained_variance   | 0.893       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0347     |
|    n_updates            | 17928       |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 0.00717     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.8e+0